In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

## Задание 1. 
### Обязательная часть
Будем парсить страницу со свежими новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [77]:
url = 'https://habr.com/ru/all/'
keywords = ['python', 'парсинг']

In [82]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')    # Получаем главную страницу с новыми статьями

In [85]:
# Задание 1.1, обязательная часть, только превью

habr_previews_1 = pd.DataFrame()

posts = soup.find_all('article', class_='post')    # Находим все статьи

for post in posts:
    if any(keyword in post.get_text().lower() for keyword in keywords):    # Проверка всех элементов поста (превью)
        title = post.find('a', class_="post__title_link").get_text()
        date = post.find('span', class_="post__time").get_text()
        link = post.find('a', class_="post__title_link").get('href')
        row = {'date': date, 'title': title, 'link':link}
        habr_previews_1 = pd.concat([habr_previews_1, pd.DataFrame([row])])
        
habr_previews_1

,date,title,link
0,вчера в 23:58,Участвуем в соревновании по Data Science. Перв...,https://habr.com/ru/post/530628/


In [86]:
# Задание 1.2, необязательная часть с полным текстом статей

habr_texts = pd.DataFrame()

posts = soup.find_all('article', class_='post')

for post in posts:
    link = post.find('a', class_="post__title_link").get('href')        # Сначала получим ссылку на полный текст
    time.sleep(0.5)    # Добавим задержку
    soup_test = BeautifulSoup(requests.get(link).text, 'html.parser')    # Сходим по ссылке
    text = soup_test.find('div', class_="post__text").get_text()        # Получим полный текст
    
    # Проверим все элементы превью, а также полный текст, на вхождение ключевых слов
    if any(keyword in post.get_text().lower() for keyword in keywords)\
    or any(keyword in text.lower() for keyword in keywords):
        title = post.find('a', class_="post__title_link").get_text()
        date = post.find('span', class_="post__time").get_text()
        row = {'date': date, 'title': title, 'link':link, 'text': text}
        habr_texts = pd.concat([habr_texts, pd.DataFrame([row])])
        
habr_texts

,date,title,link,text
0,вчера в 23:58,Участвуем в соревновании по Data Science. Перв...,https://habr.com/ru/post/530628/,"Привет, Хабр!\n\r\nДавно я не писал никаких ст..."
0,вчера в 22:15,Новые ограничения в использовании Docker Hub и...,https://habr.com/ru/company/gitlab/blog/530666/,"Ни для кого уже не новость, что начиная с 2 но..."
0,вчера в 19:23,ESP32 в окружении VSCode,https://habr.com/ru/post/530638/,В нескольких следующих статьях я хотел бы дета...
0,вчера в 18:30,Программа SmartData 2020,https://habr.com/ru/company/jugru/blog/530584/,"\nМы уже рассказывали Хабру, что новая SmartDa..."
0,вчера в 16:07,CTF-соревнования 2020 для «белых хакеров». Ста...,https://habr.com/ru/company/otus/blog/530612/,\n\n\nВ декабре OTUS при поддержке VolgaCTF и ...


In [78]:
# Можно пройти по всем страницам, сформировав их список таким образом, например:
all_pages = ['https://habr.com/ru/all/page' + str(i) + '/' for i in range(2, 51)]
all_pages.insert(0, url)
all_pages[0:5]

['https://habr.com/ru/all/',
 'https://habr.com/ru/all/page2/',
 'https://habr.com/ru/all/page3/',
 'https://habr.com/ru/all/page4/',
 'https://habr.com/ru/all/page5/']

In [58]:
# Все страницы, только превью

habr_previews = pd.DataFrame()

for page_url in all_pages:
    time.sleep(0.5)
    soup = BeautifulSoup(requests.get(page_url).text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        if any(keyword in post.get_text().lower() for keyword in keywords):
            title = post.find('a', class_="post__title_link").get_text()
            date = post.find('span', class_="post__time").get_text()
            link = post.find('a', class_="post__title_link").get('href')
            row = {'date': date, 'title': title, 'link':link}
            habr_previews = pd.concat([habr_previews, pd.DataFrame([row])])
habr_previews

,date,title,link
0,вчера в 23:58,Участвуем в соревновании по Data Science. Перв...,https://habr.com/ru/post/530628/
0,вчера в 22:10,"Регламенты закупок: кто виноват, что делать… K...",https://habr.com/ru/post/530668/
0,вчера в 18:30,Программа SmartData 2020,https://habr.com/ru/company/jugru/blog/530584/
0,вчера в 15:58,Как разработать ансамбль Light Gradient Booste...,https://habr.com/ru/company/skillfactory/blog/...
0,вчера в 13:59,"Snowflake, Anchor Model, ELT и как с этим жить",https://habr.com/ru/company/manychat/blog/530054/
...,...,...,...
0,13 ноября 2020 в 11:16,Что в контенте тебе моем? Многовековая эволюци...,https://habr.com/ru/company/skillbox/blog/527868/
0,13 ноября 2020 в 10:24,Умная нормализация данных: категориальные и по...,https://habr.com/ru/post/527860/
0,13 ноября 2020 в 09:30,Обзор операторов PostgreSQL для Kubernetes. Ча...,https://habr.com/ru/company/flant/blog/527524/
0,12 ноября 2020 в 19:48,"Константин Смирнов: «Барон контракт подписал, ...",https://habr.com/ru/company/dataart/blog/527794/


## Задание 2.
### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/). Список email-ов задаем переменной в начале кода:  
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [146]:
emails = ["xxx@x.ru", "yyy@y.com"]
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [147]:
# Столкнувшись с несколькими ошибками, скопировал все хедеры из запроса
headers = {
    'Connection': 'keep-alive',
    'Content-Length': '44',
    'Accept': 'application/json, text/plain, */*',
    'vaar-version': '0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Content-Type': 'application/json;charset=UTF-8',
    'Origin': 'https://www.avast.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.avast.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'   
}

In [148]:
# Похоже, сайт принимает на вход json; без него не получается
import json

In [149]:
payload = {'emailAddresses': emails}
req = requests.post(url, headers=headers, data=json.dumps(payload))

In [150]:
req

<Response [200]>

In [151]:
req.json()['summary']    # Перечень утечек для адресов - в 'summary'

{'xxx@x.ru': {'breaches': [3176, 12, 3, 3164, 2961, 15]},
 'yyy@y.com': {'breaches': [16613,
   16488,
   18155,
   17110,
   17670,
   13094,
   13254,
   16768,
   41,
   2,
   3587,
   17009,
   16802,
   3,
   13662,
   3520,
   15,
   3669]}}

In [152]:
req.json()['breaches']['3176']    # Подробное описание утечки по id, пример

{'breachId': 3176,
 'site': 'parapa.mail.ru',
 'recordsCount': 5029003,
 'description': "In July and August 2016, two criminals executed attacks against three separate forums hosted by Mail.ru including the Russian forum Parapa. Shortly after the breach occurred, the contents of Parapa's database were leaked publicly. The database contains usernames, email addresses, and hashed passwords for around 5 million users.",
 'publishDate': '2017-02-14T00:00:00Z',
 'statistics': {'usernames': 5029000, 'passwords': 5029003, 'emails': 4941344}}

In [154]:
email_breaches = pd.DataFrame()

for email, ids in req.json()['summary'].items():
    breach_ids = ids['breaches']    # Список id утечек из json 'summary'
    
    # Если список утечек для адреса не пустой, вытаскиваем подробную информацию из json для каждого id
    if breach_ids:
        for breach_id in breach_ids:
            breach = req.json()['breaches'][str(breach_id)]
            row = {'email': email, 'source': breach['site'], 
                   'date': breach['publishDate'], 'description': breach['description']}
            email_breaches = pd.concat([email_breaches, pd.DataFrame([row])])

email_breaches

,email,source,date,description
0,xxx@x.ru,parapa.mail.ru,2017-02-14T00:00:00Z,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,vk.com,2016-10-29T00:00:00Z,Popular Russian social networking platform VKo...
0,xxx@x.ru,adobe.com,2016-10-21T00:00:00Z,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,cfire.mail.ru,2017-02-14T00:00:00Z,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,cdprojektred.com,2017-01-31T00:00:00Z,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,imesh.com,2016-10-23T00:00:00Z,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,verifications.io,2019-03-28T00:00:00Z,Big data e-mail verification platform verifica...
0,yyy@y.com,www.dangdang.com,2019-02-21T00:00:00Z,"This is a list of email addresses only, and as..."
0,yyy@y.com,123rf.com,2020-11-19T00:00:00Z,"In March 2020, the stock image agency 123RF wa..."
0,yyy@y.com,azcentral.com,2020-01-03T00:00:00Z,"At an unconfirmed date, online Arizona newspap..."


### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: [https://vk.com/dev/methods](https://vk.com/dev/methods) , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>